In [ ]:
#@title Copyright 2019 Google LLC. { display-mode: "form" }
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="ee-notebook-buttons" align="left"><td>
<a target="_blank"  href="http://colab.research.google.com/github/google/earthengine-community/blob/master/guides/linked/ee-api-colab-setup.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a>
</td><td>
<a target="_blank"  href="https://github.com/google/earthengine-community/blob/master/guides/linked/ee-api-colab-setup.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td></table>

# Earth Engine Python API Colab Setup

This notebook demonstrates how to setup the Earth Engine Python API in Colab and provides several examples of how to print and visualize Earth Engine processed data.

## Import API and get credentials

The Earth Engine API is installed by default in Google Colaboratory so requires only importing and authenticating. These steps must be completed for each new Colab session, if you restart your Colab kernel, or if your Colab virtual machine is recycled due to inactivity.

### Import the API

Run the following cell to import the API into your session.

In [26]:
import ee
import pandas as pd
import numpy as np
import datetime
import os
from dateutil.relativedelta import relativedelta
from google.colab import files
import matplotlib.pyplot as plt
import folium

### Authenticate and initialize

Run the `ee.Authenticate` function to authenticate your access to Earth Engine servers and `ee.Initialize` to initialize it. Upon running the following cell you'll be asked to grant Earth Engine access to your Google account. Follow the instructions printed to the cell.

In [41]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='gen-lang-client-0253961861')

V2

In [50]:
def parse_date(date_str):
    """Convierte una cadena de fecha en formato YYYY-MM-DD a objeto datetime."""
    try:
        return datetime.datetime.strptime(date_str, '%Y-%m-%d')
    except ValueError:
        raise ValueError("El formato de fecha debe ser YYYY-MM-DD")

def get_date_ranges(start_date, end_date, frequency, max_chunk_days=30):
    """
    Genera rangos de fechas según la frecuencia especificada, divididos en chunks para evitar problemas de memoria.

    Args:
        start_date (datetime): Fecha de inicio
        end_date (datetime): Fecha de fin
        frequency (str): 'daily', 'monthly', o 'annual'
        max_chunk_days (int): Número máximo de días por chunk para frecuencia diaria

    Returns:
        list: Lista de tuplas (fecha_inicio, fecha_fin) para cada período
    """
    date_ranges = []

    if frequency == 'daily':
        # Para frecuencia diaria, dividir en chunks para evitar problemas de memoria
        current = start_date
        while current <= end_date:
            chunk_end = min(current + datetime.timedelta(days=max_chunk_days-1), end_date)
            date_ranges.append((current.strftime('%Y-%m-%d'), chunk_end.strftime('%Y-%m-%d')))
            current = chunk_end + datetime.timedelta(days=1)

    elif frequency == 'monthly':
        current = datetime.datetime(start_date.year, start_date.month, 1)
        while current <= end_date:
            next_month = current + relativedelta(months=1)
            end_of_month = (next_month - datetime.timedelta(days=1))
            if end_of_month > end_date:
                end_of_month = end_date
            date_ranges.append((current.strftime('%Y-%m-%d'), end_of_month.strftime('%Y-%m-%d')))
            current = next_month

    elif frequency == 'annual':
        current_year = start_date.year
        while current_year <= end_date.year:
            year_start = max(datetime.datetime(current_year, 1, 1), start_date)
            year_end = min(datetime.datetime(current_year, 12, 31), end_date)
            date_ranges.append((year_start.strftime('%Y-%m-%d'), year_end.strftime('%Y-%m-%d')))
            current_year += 1

    return date_ranges

def test_point_data_extraction(collection, point, band_name=None, scale=500):
    """
    Prueba la extracción de datos para un punto específico y verifica si hay valores válidos.

    Args:
        collection (ee.ImageCollection): Colección de imágenes
        point (ee.Geometry.Point): Punto de interés
        band_name (str, optional): Nombre de la banda específica a extraer
        scale (int): Escala en metros

    Returns:
        tuple: (bool, int) - (True si hay datos extraíbles, número de imágenes con datos válidos)
    """
    try:
        # Verificar si la colección está vacía
        count = collection.size().getInfo()
        if count == 0:
            return False, 0

        # Limitar a 10 imágenes para la prueba
        test_collection = collection.limit(10)

        # Si se especifica una banda, verificar que exista
        if band_name:
            first_image = test_collection.first()
            if first_image is None:
                return False, 0

            available_bands = first_image.bandNames().getInfo()
            if band_name not in available_bands:
                return False, 0

            test_collection = test_collection.select(band_name)

        # Función para extraer valores y verificar si son válidos
        def check_valid_data(image):
            # Extraer valor en el punto
            value_dict = image.reduceRegion(
                reducer=ee.Reducer.first(),
                geometry=point,
                scale=scale
            )

            # Verificar si hay valores no nulos
            # Método correcto para Python: verificar si el diccionario tiene elementos después de getInfo()
            value_info = value_dict.getInfo()
            has_data = 1 if value_info and any(v is not None for v in value_info.values()) else 0

            return image.set('has_data', has_data)

        # Aplicar la función a cada imagen en la colección
        test_collection = test_collection.map(check_valid_data)

        # Contar imágenes con datos válidos
        images_with_data = test_collection.filterMetadata('has_data', 'equals', 1).size().getInfo()

        return images_with_data > 0, images_with_data

    except Exception as e:
        print(f"⚠️ Error al probar extracción de datos: {e}")
        return False, 0

def check_data_availability(point, start_date, end_date):
    """
    Verifica la disponibilidad de datos para un punto y período específicos.

    Args:
        point (ee.Geometry.Point): Punto de interés
        start_date (str): Fecha de inicio en formato YYYY-MM-DD
        end_date (str): Fecha de fin en formato YYYY-MM-DD

    Returns:
        dict: Diccionario con la disponibilidad de cada tipo de datos
    """
    availability = {
        'albedo': {'available': False, 'images': 0, 'extractable': False, 'extractable_images': 0},
        'radiation': {'available': False, 'images': 0, 'extractable': False, 'extractable_images': 0},
        'temperature': {'available': False, 'images': 0, 'extractable': False, 'extractable_images': 0},
        'wind': {'available': False, 'images': 0, 'extractable': False, 'extractable_images': 0},
        'elevation': {'available': False, 'extractable': False},
        'landcover': {'available': False, 'images': 0, 'extractable': False, 'extractable_images': 0}
    }

    try:
        # Verificar albedo
        albedo_collection = ee.ImageCollection('MODIS/061/MCD43A3') \
            .filterDate(start_date, end_date) \
            .filterBounds(point)
        albedo_count = albedo_collection.size().getInfo()
        availability['albedo']['available'] = albedo_count > 0
        availability['albedo']['images'] = albedo_count

        if albedo_count > 0:
            # Verificar si se pueden extraer datos de albedo
            has_data, valid_count = test_point_data_extraction(
                albedo_collection, point, 'Albedo_BSA_vis')
            availability['albedo']['extractable'] = has_data
            availability['albedo']['extractable_images'] = valid_count

            print(f"📊 Disponibilidad de datos de albedo: {albedo_count} imágenes en la colección, {valid_count} extraíbles para el punto")
        else:
            print(f"📊 Disponibilidad de datos de albedo: {albedo_count} imágenes")

        # Verificar radiación solar
        radiation_collection = ee.ImageCollection('MODIS/061/MCD18A1') \
            .filterDate(start_date, end_date) \
            .filterBounds(point)
        radiation_count = radiation_collection.size().getInfo()
        availability['radiation']['available'] = radiation_count > 0
        availability['radiation']['images'] = radiation_count

        if radiation_count > 0:
            # Verificar si se pueden extraer datos de radiación
            # Primero verificar qué bandas están disponibles
            first_image = radiation_collection.first()
            if first_image:
                available_bands = first_image.bandNames().getInfo()
                test_band = None
                if 'DSR' in available_bands:
                    test_band = 'DSR'
                elif 'Direct' in available_bands:
                    test_band = 'Direct'
                elif 'Diffuse' in available_bands:
                    test_band = 'Diffuse'

                if test_band:
                    has_data, valid_count = test_point_data_extraction(
                        radiation_collection, point, test_band)
                    availability['radiation']['extractable'] = has_data
                    availability['radiation']['extractable_images'] = valid_count

                    print(f"📊 Disponibilidad de datos de radiación solar: {radiation_count} imágenes en la colección, {valid_count} extraíbles para el punto")
                else:
                    print(f"📊 Disponibilidad de datos de radiación solar: {radiation_count} imágenes, pero no se encontraron bandas esperadas")
            else:
                print(f"📊 Disponibilidad de datos de radiación solar: {radiation_count} imágenes, pero no se pudo acceder a la primera imagen")
        else:
            print(f"📊 Disponibilidad de datos de radiación solar: {radiation_count} imágenes")

        # Verificar temperatura
        temperature_collection = ee.ImageCollection('MODIS/061/MOD11A1') \
            .filterDate(start_date, end_date) \
            .filterBounds(point)
        temperature_count = temperature_collection.size().getInfo()
        availability['temperature']['available'] = temperature_count > 0
        availability['temperature']['images'] = temperature_count

        if temperature_count > 0:
            # Verificar si se pueden extraer datos de temperatura
            has_data, valid_count = test_point_data_extraction(
                temperature_collection, point, 'LST_Day_1km')
            availability['temperature']['extractable'] = has_data
            availability['temperature']['extractable_images'] = valid_count

            print(f"📊 Disponibilidad de datos de temperatura: {temperature_count} imágenes en la colección, {valid_count} extraíbles para el punto")
        else:
            print(f"📊 Disponibilidad de datos de temperatura: {temperature_count} imágenes")

        # Verificar viento
        wind_collection = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY') \
            .filterDate(start_date, end_date) \
            .filterBounds(point) \
            .limit(24)  # Solo verificar un día (24 horas)
        wind_count = wind_collection.size().getInfo()
        availability['wind']['available'] = wind_count > 0
        availability['wind']['images'] = wind_count

        if wind_count > 0:
            # Verificar si se pueden extraer datos de viento
            has_data, valid_count = test_point_data_extraction(
                wind_collection, point, 'u_component_of_wind_10m')
            availability['wind']['extractable'] = has_data
            availability['wind']['extractable_images'] = valid_count

            print(f"📊 Disponibilidad de datos de viento: {wind_count} imágenes en la colección, {valid_count} extraíbles para el punto")
        else:
            print(f"📊 Disponibilidad de datos de viento: {wind_count} imágenes")

        # Verificar elevación (siempre disponible globalmente)
        elevation = ee.Image('USGS/SRTMGL1_003').select('elevation')
        elevation_value = elevation.reduceRegion(
            reducer=ee.Reducer.first(),
            geometry=point,
            scale=30
        ).getInfo()
        availability['elevation']['available'] = 'elevation' in elevation_value and elevation_value['elevation'] is not None
        availability['elevation']['extractable'] = availability['elevation']['available']

        print(f"📊 Disponibilidad de datos de elevación: {'Sí' if availability['elevation']['available'] else 'No'}")

        # Verificar cobertura terrestre
        year = int(start_date.split('-')[0])
        landcover_collection = ee.ImageCollection('MODIS/006/MCD12Q1') \
            .filterDate(f"{year}-01-01", f"{year}-12-31") \
            .filterBounds(point)
        landcover_count = landcover_collection.size().getInfo()
        availability['landcover']['available'] = landcover_count > 0
        availability['landcover']['images'] = landcover_count

        if landcover_count > 0:
            # Verificar si se pueden extraer datos de cobertura terrestre
            has_data, valid_count = test_point_data_extraction(
                landcover_collection, point, 'LC_Type1')
            availability['landcover']['extractable'] = has_data
            availability['landcover']['extractable_images'] = valid_count

            print(f"📊 Disponibilidad de datos de cobertura terrestre: {landcover_count} imágenes en la colección, {valid_count} extraíbles para el punto")
        else:
            print(f"📊 Disponibilidad de datos de cobertura terrestre: {landcover_count} imágenes")

        # Resumen general
        available_count = sum(1 for v in availability.values() if v['available'])
        extractable_count = sum(1 for v in availability.values() if v['extractable'])

        print(f"\n📋 Resumen de disponibilidad:")
        print(f"  - {available_count}/6 tipos de datos disponibles en las colecciones")
        print(f"  - {extractable_count}/6 tipos de datos extraíbles para el punto específico")

        if extractable_count == 0:
            print("\n⚠️ ADVERTENCIA: No hay datos extraíbles para esta ubicación y período.")
            print("⚠️ Sugerencias:")
            print("  - Prueba con un período más reciente (2021-2022)")
            print("  - Verifica las coordenadas (algunas regiones tienen menos cobertura)")
            print("  - Intenta con una ubicación diferente")
        elif extractable_count < 3:
            print("\n⚠️ ADVERTENCIA: Pocos tipos de datos extraíbles para esta ubicación y período.")
            print("⚠️ El CSV resultante tendrá información limitada.")

    except Exception as e:
        print(f"❌ Error al verificar disponibilidad de datos: {e}")

    return availability

def check_available_bands(image_collection, expected_bands):
    """
    Verifica si las bandas esperadas están disponibles en la colección de imágenes.

    Args:
        image_collection (ee.ImageCollection): Colección de imágenes
        expected_bands (list): Lista de nombres de bandas esperadas

    Returns:
        tuple: (bool, list) - (True si todas las bandas están disponibles, lista de bandas disponibles)
    """
    try:
        # Obtener la primera imagen de la colección
        first_image = image_collection.first()
        if first_image is None:
            print("⚠️ Advertencia: Colección de imágenes vacía")
            return False, []

        # Obtener las bandas disponibles
        available_bands = first_image.bandNames().getInfo()
        if not available_bands:
            print("⚠️ Advertencia: No se pudieron obtener los nombres de las bandas")
            return False, []

        # Verificar si todas las bandas esperadas están disponibles
        all_available = all(band in available_bands for band in expected_bands)

        if not all_available:
            missing_bands = [band for band in expected_bands if band not in available_bands]
            print(f"⚠️ Advertencia: Algunas bandas esperadas no están disponibles: {missing_bands}")
            print(f"⚠️ Bandas disponibles: {available_bands}")

        return all_available, available_bands
    except Exception as e:
        print(f"⚠️ Error al verificar bandas disponibles: {e}")
        return False, []

def get_albedo_data(point, start_date, end_date):
    """
    Extrae datos de albedo (black-sky y white-sky) para un punto y período específicos.

    Args:
        point (ee.Geometry.Point): Punto de interés
        start_date (str): Fecha de inicio en formato YYYY-MM-DD
        end_date (str): Fecha de fin en formato YYYY-MM-DD

    Returns:
        ee.ImageCollection: Colección de imágenes con datos de albedo
    """
    albedo_collection = ee.ImageCollection('MODIS/061/MCD43A3') \
        .filterDate(start_date, end_date) \
        .filterBounds(point)

    # Verificar bandas disponibles
    expected_bands = ['Albedo_BSA_vis', 'Albedo_WSA_vis', 'Albedo_BSA_nir', 'Albedo_WSA_nir', 'Albedo_BSA_shortwave', 'Albedo_WSA_shortwave']
    bands_available, available_bands = check_available_bands(albedo_collection, expected_bands)

    if bands_available:
        albedo_collection = albedo_collection.select(expected_bands)
    else:
        print("⚠️ No se pudieron seleccionar todas las bandas de albedo esperadas")
        # Seleccionar solo las bandas disponibles que coinciden con el patrón esperado
        available_albedo_bands = [band for band in available_bands if 'Albedo' in band]
        if available_albedo_bands:
            print(f"🔄 Seleccionando bandas de albedo disponibles: {available_albedo_bands}")
            albedo_collection = albedo_collection.select(available_albedo_bands)

    return albedo_collection

def get_solar_radiation_data(point, start_date, end_date):
    """
    Extrae datos de radiación solar para un punto y período específicos.

    Args:
        point (ee.Geometry.Point): Punto de interés
        start_date (str): Fecha de inicio en formato YYYY-MM-DD
        end_date (str): Fecha de fin en formato YYYY-MM-DD

    Returns:
        ee.ImageCollection: Colección de imágenes con datos de radiación solar
    """
    radiation_collection = ee.ImageCollection('MODIS/061/MCD18A1') \
        .filterDate(start_date, end_date) \
        .filterBounds(point)

    # Verificar bandas disponibles
    # Nombres de bandas actualizados según el error reportado
    expected_bands = ['DSR', 'Direct', 'Diffuse']
    bands_available, available_bands = check_available_bands(radiation_collection, expected_bands)

    if bands_available:
        radiation_collection = radiation_collection.select(expected_bands)
    else:
        print("⚠️ No se pudieron seleccionar todas las bandas de radiación solar esperadas")
        # Intentar seleccionar bandas disponibles relacionadas con radiación solar
        radiation_bands = [band for band in available_bands if 'DSR' in band or 'Direct' in band or 'Diffuse' in band]
        if radiation_bands:
            print(f"🔄 Seleccionando bandas de radiación solar disponibles: {radiation_bands}")
            radiation_collection = radiation_collection.select(radiation_bands)

    return radiation_collection

def get_temperature_data(point, start_date, end_date):
    """
    Extrae datos de temperatura superficial para un punto y período específicos.

    Args:
        point (ee.Geometry.Point): Punto de interés
        start_date (str): Fecha de inicio en formato YYYY-MM-DD
        end_date (str): Fecha de fin en formato YYYY-MM-DD

    Returns:
        ee.ImageCollection: Colección de imágenes con datos de temperatura
    """
    temperature_collection = ee.ImageCollection('MODIS/061/MOD11A1') \
        .filterDate(start_date, end_date) \
        .filterBounds(point)

    # Verificar bandas disponibles
    expected_bands = ['LST_Day_1km', 'LST_Night_1km', 'QC_Day', 'QC_Night']
    bands_available, available_bands = check_available_bands(temperature_collection, expected_bands)

    if bands_available:
        temperature_collection = temperature_collection.select(expected_bands)
    else:
        print("⚠️ No se pudieron seleccionar todas las bandas de temperatura esperadas")
        # Intentar seleccionar bandas disponibles relacionadas con temperatura
        temp_bands = [band for band in available_bands if 'LST' in band]
        if temp_bands:
            print(f"🔄 Seleccionando bandas de temperatura disponibles: {temp_bands}")
            temperature_collection = temperature_collection.select(temp_bands)

    return temperature_collection

def get_wind_data_chunk(point, start_date, end_date):
    """
    Extrae datos de viento para un punto y período específicos (un chunk).

    Args:
        point (ee.Geometry.Point): Punto de interés
        start_date (str): Fecha de inicio en formato YYYY-MM-DD
        end_date (str): Fecha de fin en formato YYYY-MM-DD

    Returns:
        ee.ImageCollection: Colección de imágenes con datos de viento
    """
    # Obtener datos de viento horarios
    wind_collection = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY') \
        .filterDate(start_date, end_date) \
        .filterBounds(point)

    # Verificar bandas disponibles
    expected_bands = ['u_component_of_wind_10m', 'v_component_of_wind_10m']
    bands_available, available_bands = check_available_bands(wind_collection, expected_bands)

    if bands_available:
        wind_collection = wind_collection.select(expected_bands)
    else:
        print("⚠️ No se pudieron seleccionar todas las bandas de viento esperadas")
        # Intentar seleccionar bandas disponibles relacionadas con viento
        wind_bands = [band for band in available_bands if 'wind' in band.lower()]
        if wind_bands:
            print(f"🔄 Seleccionando bandas de viento disponibles: {wind_bands}")
            wind_collection = wind_collection.select(wind_bands)

    # Método alternativo para agrupar por día sin usar startOfDay()
    # Usamos format() para truncar la hora y obtener solo la fecha
    def add_date_band(image):
        # Obtener la fecha como string en formato YYYY-MM-DD
        date_string = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
        # Añadir como propiedad
        return image.set('date_string', date_string)

    # Añadir la propiedad de fecha a cada imagen
    wind_collection_with_date = wind_collection.map(add_date_band)

    # Obtener lista de fechas únicas como strings
    unique_dates = wind_collection_with_date.aggregate_array('date_string').distinct()

    # Función para calcular la media diaria
    def calculate_daily_mean(date_string):
        # Filtrar imágenes para esta fecha
        daily_images = wind_collection_with_date.filter(ee.Filter.eq('date_string', date_string))
        # Calcular la media
        mean_image = daily_images.mean()
        # Convertir la fecha string a timestamp
        timestamp = ee.Date(date_string).millis()
        # Añadir timestamp y fecha string como propiedades
        return mean_image.set('system:time_start', timestamp).set('date_string', date_string)

    # Calcular medias diarias
    daily_wind = ee.ImageCollection.fromImages(unique_dates.map(calculate_daily_mean))

    return daily_wind

def get_elevation_data(point):
    """
    Extrae datos de elevación y variables topográficas derivadas para un punto específico.

    Args:
        point (ee.Geometry.Point): Punto de interés

    Returns:
        ee.Image: Imagen con datos de elevación y variables derivadas
    """
    # Obtener datos de elevación SRTM
    elevation = ee.Image('USGS/SRTMGL1_003').select('elevation')

    # Calcular pendiente y aspecto (orientación)
    slope = ee.Terrain.slope(elevation)
    aspect = ee.Terrain.aspect(elevation)

    # Combinar en una sola imagen
    topo = elevation.addBands(slope).addBands(aspect).rename(['elevation', 'slope', 'aspect'])

    return topo

def get_landcover_data(point, year):
    """
    Extrae datos de cobertura terrestre para un punto y año específicos.

    Args:
        point (ee.Geometry.Point): Punto de interés
        year (int): Año para el cual obtener la cobertura terrestre

    Returns:
        ee.Image: Imagen con datos de cobertura terrestre
    """
    # Usar el producto de cobertura terrestre MODIS
    landcover_collection = ee.ImageCollection('MODIS/006/MCD12Q1') \
        .filterBounds(point)

    # Filtrar por el año especificado
    year_start = f"{year}-01-01"
    year_end = f"{year}-12-31"
    landcover = landcover_collection.filterDate(year_start, year_end).first()

    if landcover is None:
        print(f"⚠️ Advertencia: No hay datos de cobertura terrestre disponibles para el año {year}")
        # Intentar con años anteriores si no hay datos para el año especificado
        for prev_year in range(year-1, year-5, -1):
            print(f"🔄 Intentando con datos de cobertura terrestre del año {prev_year}...")
            prev_year_start = f"{prev_year}-01-01"
            prev_year_end = f"{prev_year}-12-31"
            landcover = landcover_collection.filterDate(prev_year_start, prev_year_end).first()
            if landcover is not None:
                print(f"✅ Se encontraron datos de cobertura terrestre para el año {prev_year}")
                break

        if landcover is None:
            return None

    # Verificar bandas disponibles
    expected_bands = ['LC_Type1']
    bands_available, available_bands = check_available_bands(landcover_collection, expected_bands)

    if bands_available:
        landcover = landcover.select('LC_Type1')
    else:
        print("⚠️ No se pudieron seleccionar las bandas de cobertura terrestre esperadas")
        # Intentar seleccionar bandas disponibles relacionadas con cobertura terrestre
        lc_bands = [band for band in available_bands if 'LC' in band or 'Type' in band]
        if lc_bands:
            print(f"🔄 Seleccionando bandas de cobertura terrestre disponibles: {lc_bands}")
            landcover = landcover.select(lc_bands[0])  # Seleccionar la primera banda disponible

    return landcover

def extract_point_values(image_collection, point, scale=500, band_name=None):
    """
    Extrae valores para un punto específico de una colección de imágenes.

    Args:
        image_collection (ee.ImageCollection): Colección de imágenes
        point (ee.Geometry.Point): Punto de interés
        scale (int): Escala en metros
        band_name (str, optional): Nombre de la banda específica a extraer

    Returns:
        list: Lista de diccionarios con fecha y valores
    """
    # Verificar si la colección está vacía
    count = image_collection.size().getInfo()
    if count == 0:
        print("⚠️ Advertencia: Colección de imágenes vacía")
        return []

    # Si se especifica una banda, verificar que exista
    if band_name:
        first_image = image_collection.first()
        if first_image is None:
            print("⚠️ Advertencia: No hay imágenes en la colección")
            return []

        available_bands = first_image.bandNames().getInfo()
        if band_name not in available_bands:
            print(f"⚠️ Advertencia: La banda '{band_name}' no está disponible. Bandas disponibles: {available_bands}")
            return []

    def extract_from_image(image):
        try:
            # Si se especifica una banda, seleccionarla
            if band_name:
                image = image.select(band_name)

            # Extraer valor en el punto
            value = image.reduceRegion(
                reducer=ee.Reducer.first(),
                geometry=point,
                scale=scale
            )

            # Crear feature con fecha y valor
            return ee.Feature(None, {
                'date': ee.Date(image.get('system:time_start')).format('YYYY-MM-dd'),
                'values': value
            })
        except Exception as e:
            print(f"⚠️ Error al extraer valores de la imagen: {e}")
            return ee.Feature(None, {
                'date': ee.Date(image.get('system:time_start')).format('YYYY-MM-dd'),
                'values': {}
            })

    try:
        # Aplicar la función a cada imagen en la colección
        features = image_collection.map(extract_from_image)

        # Convertir a lista para descargar
        result = features.aggregate_array('properties').getInfo()

        # Filtrar resultados para eliminar valores vacíos
        filtered_result = []
        for item in result:
            if item['values'] and len(item['values']) > 0 and any(v is not None for v in item['values'].values()):
                filtered_result.append(item)

        if len(filtered_result) == 0:
            print(f"⚠️ No se encontraron valores válidos para el punto en ninguna de las {count} imágenes")
        else:
            print(f"✅ Se encontraron valores válidos en {len(filtered_result)} de {count} imágenes")

        return filtered_result
    except Exception as e:
        print(f"⚠️ Error al extraer valores de la colección: {e}")
        return []

def extract_static_values(image, point, scale=500):
    """
    Extrae valores para un punto específico de una imagen estática.

    Args:
        image (ee.Image): Imagen
        point (ee.Geometry.Point): Punto de interés
        scale (int): Escala en metros

    Returns:
        dict: Diccionario con valores
    """
    if image is None:
        print("⚠️ Advertencia: Imagen nula")
        return {}

    try:
        values = image.reduceRegion(
            reducer=ee.Reducer.first(),
            geometry=point,
            scale=scale
        ).getInfo()

        # Verificar si se obtuvieron valores válidos
        if not values or all(v is None for v in values.values()):
            print("⚠️ No se encontraron valores válidos para el punto en la imagen estática")
            return {}

        return values
    except Exception as e:
        print(f"⚠️ Error al extraer valores estáticos: {e}")
        return {}

def safe_merge(df_list):
    """
    Combina de forma segura una lista de DataFrames, verificando que tengan la columna 'date'.

    Args:
        df_list (list): Lista de DataFrames a combinar

    Returns:
        pd.DataFrame: DataFrame combinado o DataFrame vacío con columna 'date'
    """
    # Filtrar DataFrames vacíos o sin columna 'date'
    valid_dfs = [df for df in df_list if not df.empty and 'date' in df.columns]

    if not valid_dfs:
        # Retornar DataFrame vacío con columna 'date'
        return pd.DataFrame({'date': []})

    # Comenzar con el primer DataFrame válido
    result = valid_dfs[0].copy()

    # Combinar con el resto de DataFrames válidos
    for df in valid_dfs[1:]:
        result = pd.merge(result, df, on='date', how='outer')

    return result

def process_albedo_data(albedo_collection, point, frequency, start_date, end_date):
    """
    Procesa datos de albedo y los agrega según la frecuencia especificada.

    Args:
        albedo_collection (ee.ImageCollection): Colección de imágenes de albedo
        point (ee.Geometry.Point): Punto de interés
        frequency (str): 'daily', 'monthly', o 'annual'
        start_date (datetime): Fecha de inicio
        end_date (datetime): Fecha de fin

    Returns:
        list: Lista de diccionarios con datos de albedo agregados
    """
    # Verificar bandas disponibles
    first_image = albedo_collection.first()
    if first_image is None:
        print("⚠️ Advertencia: No hay imágenes de albedo disponibles")
        return []

    available_bands = first_image.bandNames().getInfo()

    # Extraer valores para cada banda disponible
    valid_dfs = []

    # Bandas BSA (Black-Sky Albedo)
    bsa_bands = [band for band in available_bands if 'BSA' in band]
    for band in bsa_bands:
        try:
            data = extract_point_values(albedo_collection.select(band), point)
            if data:
                df_name = band.lower().replace('albedo_', '')
                df = pd.DataFrame([{'date': item['date'], df_name: item['values'].get(band, None)} for item in data])
                if not df.empty and 'date' in df.columns:
                    # Convertir valores a escala correcta (dividir por 1000)
                    df[df_name] = df[df_name].apply(lambda x: x / 1000.0 if x is not None else None)
                    valid_dfs.append(df)
        except Exception as e:
            print(f"⚠️ Error al procesar datos de {band}: {e}")

    # Bandas WSA (White-Sky Albedo)
    wsa_bands = [band for band in available_bands if 'WSA' in band]
    for band in wsa_bands:
        try:
            data = extract_point_values(albedo_collection.select(band), point)
            if data:
                df_name = band.lower().replace('albedo_', '')
                df = pd.DataFrame([{'date': item['date'], df_name: item['values'].get(band, None)} for item in data])
                if not df.empty and 'date' in df.columns:
                    # Convertir valores a escala correcta (dividir por 1000)
                    df[df_name] = df[df_name].apply(lambda x: x / 1000.0 if x is not None else None)
                    valid_dfs.append(df)
        except Exception as e:
            print(f"⚠️ Error al procesar datos de {band}: {e}")

    # Verificar si hay datos disponibles
    if not valid_dfs:
        print("⚠️ Advertencia: No hay datos de albedo disponibles para el período seleccionado")
        return []

    # Combinar DataFrames de forma segura
    albedo_df = safe_merge(valid_dfs)

    # Si el DataFrame resultante está vacío o no tiene columna 'date', retornar lista vacía
    if albedo_df.empty or 'date' not in albedo_df.columns:
        print("⚠️ El DataFrame combinado de albedo está vacío o no tiene columna 'date'")
        return []

    # Convertir fechas a datetime
    albedo_df['date'] = pd.to_datetime(albedo_df['date'])

    # Agregar según frecuencia
    if frequency == 'daily':
        return albedo_df.to_dict('records')
    elif frequency == 'monthly':
        albedo_df.set_index('date', inplace=True)
        monthly_df = albedo_df.resample('M').mean()
        monthly_df.index = monthly_df.index.strftime('%Y-%m-%d')
        return monthly_df.reset_index().to_dict('records')
    elif frequency == 'annual':
        albedo_df.set_index('date', inplace=True)
        annual_df = albedo_df.resample('Y').mean()
        annual_df.index = annual_df.index.strftime('%Y-%m-%d')
        return annual_df.reset_index().to_dict('records')

def process_radiation_data(radiation_collection, point, frequency, start_date, end_date):
    """
    Procesa datos de radiación solar y los agrega según la frecuencia especificada.

    Args:
        radiation_collection (ee.ImageCollection): Colección de imágenes de radiación
        point (ee.Geometry.Point): Punto de interés
        frequency (str): 'daily', 'monthly', o 'annual'
        start_date (datetime): Fecha de inicio
        end_date (datetime): Fecha de fin

    Returns:
        list: Lista de diccionarios con datos de radiación agregados
    """
    # Verificar bandas disponibles
    first_image = radiation_collection.first()
    if first_image is None:
        print("⚠️ Advertencia: No hay imágenes de radiación solar disponibles")
        return []

    available_bands = first_image.bandNames().getInfo()

    # Extraer valores para cada banda disponible
    valid_dfs = []

    for band in available_bands:
        try:
            data = extract_point_values(radiation_collection.select(band), point)
            if data:
                df_name = f"radiation_{band.lower()}"
                df = pd.DataFrame([{'date': item['date'], df_name: item['values'].get(band, None)} for item in data])
                if not df.empty and 'date' in df.columns:
                    valid_dfs.append(df)
        except Exception as e:
            print(f"⚠️ Error al procesar datos de radiación {band}: {e}")

    # Verificar si hay datos disponibles
    if not valid_dfs:
        print("⚠️ Advertencia: No hay datos de radiación solar disponibles para el período seleccionado")
        return []

    # Combinar DataFrames de forma segura
    radiation_df = safe_merge(valid_dfs)

    # Si el DataFrame resultante está vacío o no tiene columna 'date', retornar lista vacía
    if radiation_df.empty or 'date' not in radiation_df.columns:
        print("⚠️ El DataFrame combinado de radiación está vacío o no tiene columna 'date'")
        return []

    # Convertir fechas a datetime
    radiation_df['date'] = pd.to_datetime(radiation_df['date'])

    # Agregar según frecuencia
    if frequency == 'daily':
        return radiation_df.to_dict('records')
    elif frequency == 'monthly':
        radiation_df.set_index('date', inplace=True)
        monthly_df = radiation_df.resample('M').mean()
        monthly_df.index = monthly_df.index.strftime('%Y-%m-%d')
        return monthly_df.reset_index().to_dict('records')
    elif frequency == 'annual':
        radiation_df.set_index('date', inplace=True)
        annual_df = radiation_df.resample('Y').mean()
        annual_df.index = annual_df.index.strftime('%Y-%m-%d')
        return annual_df.reset_index().to_dict('records')

def process_temperature_data(temperature_collection, point, frequency, start_date, end_date):
    """
    Procesa datos de temperatura y los agrega según la frecuencia especificada.

    Args:
        temperature_collection (ee.ImageCollection): Colección de imágenes de temperatura
        point (ee.Geometry.Point): Punto de interés
        frequency (str): 'daily', 'monthly', o 'annual'
        start_date (datetime): Fecha de inicio
        end_date (datetime): Fecha de fin

    Returns:
        list: Lista de diccionarios con datos de temperatura agregados
    """
    # Verificar bandas disponibles
    first_image = temperature_collection.first()
    if first_image is None:
        print("⚠️ Advertencia: No hay imágenes de temperatura disponibles")
        return []

    available_bands = first_image.bandNames().getInfo()

    # Extraer valores para bandas de temperatura
    valid_dfs = []
    temp_bands = [band for band in available_bands if 'LST' in band]

    for band in temp_bands:
        try:
            data = extract_point_values(temperature_collection.select(band), point)
            if data:
                df_name = f"temp_{band.lower().replace('lst_', '')}"
                df = pd.DataFrame([{'date': item['date'], df_name: item['values'].get(band, None)} for item in data])
                if not df.empty and 'date' in df.columns:
                    # Convertir valores a escala correcta (convertir a Celsius)
                    df[df_name] = df[df_name].apply(lambda x: (x * 0.02) - 273.15 if x is not None else None)
                    valid_dfs.append(df)
        except Exception as e:
            print(f"⚠️ Error al procesar datos de temperatura {band}: {e}")

    # Verificar si hay datos disponibles
    if not valid_dfs:
        print("⚠️ Advertencia: No hay datos de temperatura disponibles para el período seleccionado")
        return []

    # Combinar DataFrames de forma segura
    temp_df = safe_merge(valid_dfs)

    # Si el DataFrame resultante está vacío o no tiene columna 'date', retornar lista vacía
    if temp_df.empty or 'date' not in temp_df.columns:
        print("⚠️ El DataFrame combinado de temperatura está vacío o no tiene columna 'date'")
        return []

    # Añadir temperatura media si hay datos de día y noche
    day_col = next((col for col in temp_df.columns if 'day' in col.lower()), None)
    night_col = next((col for col in temp_df.columns if 'night' in col.lower()), None)

    if day_col and night_col:
        temp_df['temp_mean'] = temp_df[[day_col, night_col]].mean(axis=1)
    elif day_col:
        temp_df['temp_mean'] = temp_df[day_col]
    elif night_col:
        temp_df['temp_mean'] = temp_df[night_col]

    # Convertir fechas a datetime
    temp_df['date'] = pd.to_datetime(temp_df['date'])

    # Agregar según frecuencia
    if frequency == 'daily':
        return temp_df.to_dict('records')
    elif frequency == 'monthly':
        temp_df.set_index('date', inplace=True)
        monthly_df = temp_df.resample('M').mean()
        monthly_df.index = monthly_df.index.strftime('%Y-%m-%d')
        return monthly_df.reset_index().to_dict('records')
    elif frequency == 'annual':
        temp_df.set_index('date', inplace=True)
        annual_df = temp_df.resample('Y').mean()
        annual_df.index = annual_df.index.strftime('%Y-%m-%d')
        return annual_df.reset_index().to_dict('records')

def process_wind_data_chunked(point, start_date, end_date, frequency, max_chunk_days=30):
    """
    Procesa datos de viento en chunks para evitar problemas de memoria.

    Args:
        point (ee.Geometry.Point): Punto de interés
        start_date (datetime): Fecha de inicio
        end_date (datetime): Fecha de fin
        frequency (str): 'daily', 'monthly', o 'annual'
        max_chunk_days (int): Número máximo de días por chunk

    Returns:
        list: Lista de diccionarios con datos de viento agregados
    """
    # Generar rangos de fechas en chunks
    date_ranges = get_date_ranges(start_date, end_date, 'daily', max_chunk_days)

    # Procesar cada chunk
    all_wind_data = []

    for i, (chunk_start, chunk_end) in enumerate(date_ranges):
        print(f"🔄 Procesando chunk de viento {i+1}/{len(date_ranges)}: {chunk_start} a {chunk_end}")

        try:
            # Obtener datos para este chunk
            wind_collection = get_wind_data_chunk(point, chunk_start, chunk_end)

            # Verificar bandas disponibles
            first_image = wind_collection.first()
            if first_image is None:
                print(f"⚠️ No hay imágenes de viento disponibles para el chunk {chunk_start} a {chunk_end}")
                continue

            available_bands = first_image.bandNames().getInfo()

            # Extraer valores para cada componente
            valid_dfs = []

            # Buscar componentes U y V del viento
            u_band = next((band for band in available_bands if 'u_component' in band.lower()), None)
            v_band = next((band for band in available_bands if 'v_component' in band.lower()), None)

            if u_band:
                try:
                    u_data = extract_point_values(wind_collection.select(u_band), point)
                    if u_data:
                        u_df = pd.DataFrame([{'date': item['date'], 'wind_u': item['values'].get(u_band, None)} for item in u_data])
                        if not u_df.empty and 'date' in u_df.columns:
                            valid_dfs.append(u_df)
                except Exception as e:
                    print(f"⚠️ Error al procesar datos de componente U del viento: {e}")

            if v_band:
                try:
                    v_data = extract_point_values(wind_collection.select(v_band), point)
                    if v_data:
                        v_df = pd.DataFrame([{'date': item['date'], 'wind_v': item['values'].get(v_band, None)} for item in v_data])
                        if not v_df.empty and 'date' in v_df.columns:
                            valid_dfs.append(v_df)
                except Exception as e:
                    print(f"⚠️ Error al procesar datos de componente V del viento: {e}")

            # Verificar si hay datos disponibles
            if not valid_dfs:
                print(f"⚠️ No hay datos de viento disponibles para el chunk {chunk_start} a {chunk_end}")
                continue

            # Combinar DataFrames de forma segura
            wind_df = safe_merge(valid_dfs)

            # Si el DataFrame resultante está vacío o no tiene columna 'date', continuar con el siguiente chunk
            if wind_df.empty or 'date' not in wind_df.columns:
                print(f"⚠️ El DataFrame combinado de viento está vacío para el chunk {chunk_start} a {chunk_end}")
                continue

            # Calcular velocidad y dirección del viento si hay ambos componentes
            if 'wind_u' in wind_df.columns and 'wind_v' in wind_df.columns:
                wind_df['wind_speed'] = wind_df.apply(lambda row: np.sqrt(row['wind_u']**2 + row['wind_v']**2) if pd.notnull(row['wind_u']) and pd.notnull(row['wind_v']) else None, axis=1)
                wind_df['wind_direction'] = wind_df.apply(lambda row: (270 - np.degrees(np.arctan2(row['wind_v'], row['wind_u']))) % 360 if pd.notnull(row['wind_u']) and pd.notnull(row['wind_v']) else None, axis=1)

            # Convertir fechas a datetime
            wind_df['date'] = pd.to_datetime(wind_df['date'])

            # Agregar a la lista de todos los datos
            all_wind_data.append(wind_df)

            # Pausa para evitar sobrecargar la API
            time.sleep(1)

        except Exception as e:
            print(f"⚠️ Error al procesar chunk de viento {chunk_start} a {chunk_end}: {e}")
            # Continuar con el siguiente chunk
            continue

    # Verificar si hay datos disponibles
    if not all_wind_data:
        print("⚠️ Advertencia: No hay datos de viento disponibles para el período seleccionado")
        return []

    # Combinar todos los chunks
    combined_wind_df = pd.concat(all_wind_data, ignore_index=True)

    # Eliminar duplicados si los hay
    combined_wind_df.drop_duplicates(subset=['date'], keep='first', inplace=True)

    # Agregar según frecuencia
    if frequency == 'daily':
        return combined_wind_df.to_dict('records')
    elif frequency == 'monthly':
        combined_wind_df.set_index('date', inplace=True)
        monthly_df = combined_wind_df.resample('M').mean()
        monthly_df.index = monthly_df.index.strftime('%Y-%m-%d')
        return monthly_df.reset_index().to_dict('records')
    elif frequency == 'annual':
        combined_wind_df.set_index('date', inplace=True)
        annual_df = combined_wind_df.resample('Y').mean()
        annual_df.index = annual_df.index.strftime('%Y-%m-%d')
        return annual_df.reset_index().to_dict('records')

def merge_all_data(albedo_data, radiation_data, temperature_data, wind_data, topo_data, landcover_data):
    """
    Combina todos los datos en un solo DataFrame.

    Args:
        albedo_data (list): Lista de diccionarios con datos de albedo
        radiation_data (list): Lista de diccionarios con datos de radiación
        temperature_data (list): Lista de diccionarios con datos de temperatura
        wind_data (list): Lista de diccionarios con datos de viento
        topo_data (dict): Diccionario con datos topográficos
        landcover_data (dict): Diccionario con datos de cobertura terrestre

    Returns:
        pd.DataFrame: DataFrame con todos los datos combinados
    """
    # Convertir listas a DataFrames
    dfs = []

    # Solo agregar DataFrames válidos con columna 'date'
    if albedo_data:
        try:
            df = pd.DataFrame(albedo_data)
            if not df.empty and 'date' in df.columns:
                dfs.append(df)
        except Exception as e:
            print(f"⚠️ Error al convertir datos de albedo a DataFrame: {e}")

    if radiation_data:
        try:
            df = pd.DataFrame(radiation_data)
            if not df.empty and 'date' in df.columns:
                dfs.append(df)
        except Exception as e:
            print(f"⚠️ Error al convertir datos de radiación a DataFrame: {e}")

    if temperature_data:
        try:
            df = pd.DataFrame(temperature_data)
            if not df.empty and 'date' in df.columns:
                dfs.append(df)
        except Exception as e:
            print(f"⚠️ Error al convertir datos de temperatura a DataFrame: {e}")

    if wind_data:
        try:
            df = pd.DataFrame(wind_data)
            if not df.empty and 'date' in df.columns:
                dfs.append(df)
        except Exception as e:
            print(f"⚠️ Error al convertir datos de viento a DataFrame: {e}")

    # Si no hay DataFrames válidos, retornar DataFrame vacío con columna 'date'
    if not dfs:
        print("⚠️ Advertencia: No hay datos disponibles para el período seleccionado")
        return pd.DataFrame({'date': []})

    # Combinar DataFrames de forma segura
    merged_df = safe_merge(dfs)

    # Si el DataFrame resultante está vacío o no tiene columna 'date', retornar DataFrame vacío con columna 'date'
    if merged_df.empty or 'date' not in merged_df.columns:
        print("⚠️ El DataFrame combinado está vacío o no tiene columna 'date'")
        return pd.DataFrame({'date': []})

    # Añadir datos topográficos (constantes para todas las fechas)
    if topo_data:
        for key, value in topo_data.items():
            merged_df[key] = value

    # Añadir datos de cobertura terrestre (constantes para todas las fechas)
    if landcover_data:
        for key, value in landcover_data.items():
            merged_df[key] = value

    return merged_df

def visualize_location(lat, lon, zoom=10):
    """
    Visualiza la ubicación en un mapa interactivo.

    Args:
        lat (float): Latitud
        lon (float): Longitud
        zoom (int): Nivel de zoom
    """
    # Crear mapa centrado en la ubicación
    m = folium.Map(location=[lat, lon], zoom_start=zoom)

    # Añadir marcador
    folium.Marker(
        location=[lat, lon],
        popup=f"Lat: {lat}, Lon: {lon}",
        icon=folium.Icon(color="red", icon="info-sign")
    ).add_to(m)

    # Mostrar mapa
    display(m)

def plot_data(df, variable, title=None, ylabel=None):
    """
    Genera un gráfico de línea para una variable específica.

    Args:
        df (pd.DataFrame): DataFrame con los datos
        variable (str): Nombre de la columna a graficar
        title (str, optional): Título del gráfico
        ylabel (str, optional): Etiqueta del eje Y
    """
    if df.empty or variable not in df.columns:
        print(f"⚠️ No hay datos disponibles para graficar {variable}")
        return

    # Verificar que hay datos no nulos para graficar
    if df[variable].isna().all():
        print(f"⚠️ Todos los valores de {variable} son nulos, no se puede graficar")
        return

    plt.figure(figsize=(12, 6))
    plt.plot(df['date'], df[variable], marker='o', linestyle='-')
    plt.title(title or f"{variable} a lo largo del tiempo")
    plt.xlabel("Fecha")
    plt.ylabel(ylabel or variable)
    plt.grid(True, alpha=0.3)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

def validate_date_range(start_date, end_date, frequency):
    """
    Valida el rango de fechas y la frecuencia, mostrando advertencias si es necesario.

    Args:
        start_date (datetime): Fecha de inicio
        end_date (datetime): Fecha de fin
        frequency (str): Frecuencia de los datos ('daily', 'monthly', o 'annual')

    Returns:
        bool: True si el rango es válido, False si es potencialmente problemático
    """
    # Calcular la duración en días
    duration_days = (end_date - start_date).days + 1

    # Validar según la frecuencia
    if frequency == 'daily':
        if duration_days > 90:  # Más de 3 meses
            print(f"⚠️ ADVERTENCIA: Has seleccionado {duration_days} días con frecuencia diaria.")
            print("⚠️ Esto puede causar problemas de memoria o tiempos de procesamiento muy largos.")
            print("⚠️ Se recomienda reducir el rango de fechas o cambiar a frecuencia mensual.")
            return False
    elif frequency == 'monthly':
        if duration_days > 1095:  # Más de 3 años
            print(f"⚠️ ADVERTENCIA: Has seleccionado un período de más de 3 años con frecuencia mensual.")
            print("⚠️ Esto puede causar tiempos de procesamiento largos.")
            print("⚠️ Considera reducir el rango de fechas o cambiar a frecuencia anual para períodos muy largos.")
            return False

    return True

def suggest_alternative_dates(point):
    """
    Sugiere fechas alternativas con buena disponibilidad de datos.

    Args:
        point (ee.Geometry.Point): Punto de interés

    Returns:
        list: Lista de años con buena disponibilidad de datos
    """
    print("🔍 Buscando períodos con mejor disponibilidad de datos...")

    # Lista de años a verificar (más recientes primero)
    years_to_check = list(range(2022, 2015, -1))
    good_years = []

    for year in years_to_check:
        year_start = f"{year}-01-01"
        year_end = f"{year}-12-31"

        # Verificar albedo
        albedo_collection = ee.ImageCollection('MODIS/061/MCD43A3') \
            .filterDate(year_start, year_end) \
            .filterBounds(point)
        albedo_count = albedo_collection.size().getInfo()

        # Verificar si se pueden extraer datos de albedo
        albedo_extractable = False
        if albedo_count > 0:
            has_data, valid_count = test_point_data_extraction(
                albedo_collection, point, 'Albedo_BSA_vis')
            albedo_extractable = has_data

        # Verificar radiación
        radiation_collection = ee.ImageCollection('MODIS/061/MCD18A1') \
            .filterDate(year_start, year_end) \
            .filterBounds(point)
        radiation_count = radiation_collection.size().getInfo()

        # Verificar si se pueden extraer datos de radiación
        radiation_extractable = False
        if radiation_count > 0:
            first_image = radiation_collection.first()
            if first_image:
                available_bands = first_image.bandNames().getInfo()
                test_band = None
                if 'DSR' in available_bands:
                    test_band = 'DSR'
                elif 'Direct' in available_bands:
                    test_band = 'Direct'
                elif 'Diffuse' in available_bands:
                    test_band = 'Diffuse'

                if test_band:
                    has_data, valid_count = test_point_data_extraction(
                        radiation_collection, point, test_band)
                    radiation_extractable = has_data

        # Verificar temperatura
        temperature_collection = ee.ImageCollection('MODIS/061/MOD11A1') \
            .filterDate(year_start, year_end) \
            .filterBounds(point)
        temperature_count = temperature_collection.size().getInfo()

        # Verificar si se pueden extraer datos de temperatura
        temperature_extractable = False
        if temperature_count > 0:
            has_data, valid_count = test_point_data_extraction(
                temperature_collection, point, 'LST_Day_1km')
            temperature_extractable = has_data

        # Evaluar disponibilidad
        extractable_count = sum([albedo_extractable, radiation_extractable, temperature_extractable])
        if extractable_count >= 2:  # Al menos 2 de 3 tipos de datos son extraíbles
            good_years.append(year)
            print(f"✅ {year}: Buena disponibilidad de datos extraíbles")
            print(f"   - Albedo: {'✓' if albedo_extractable else '✗'}")
            print(f"   - Radiación: {'✓' if radiation_extractable else '✗'}")
            print(f"   - Temperatura: {'✓' if temperature_extractable else '✗'}")

            # Limitar a los 3 mejores años
            if len(good_years) >= 3:
                break
        else:
            print(f"❌ {year}: Disponibilidad limitada de datos extraíbles")
            print(f"   - Albedo: {'✓' if albedo_extractable else '✗'}")
            print(f"   - Radiación: {'✓' if radiation_extractable else '✗'}")
            print(f"   - Temperatura: {'✓' if temperature_extractable else '✗'}")

    if good_years:
        print(f"\n✨ Sugerencia: Prueba con el año {good_years[0]} para obtener mejores resultados.")
    else:
        print("\n⚠️ No se encontraron años con buena disponibilidad de datos extraíbles para esta ubicación.")
        print("⚠️ Considera probar con otra ubicación o utilizar fuentes de datos alternativas.")

    return good_years

def extract_solar_data_colab(lat, lon, start_date, end_date, frequency='monthly', output_filename='solar_data.csv', check_availability=True):
    """
    Función principal para extraer datos solares en Google Colab.

    Args:
        lat (float): Latitud del punto de interés
        lon (float): Longitud del punto de interés
        start_date (str): Fecha de inicio en formato YYYY-MM-DD
        end_date (str): Fecha de fin en formato YYYY-MM-DD
        frequency (str): Frecuencia de los datos ('daily', 'monthly', o 'annual')
        output_filename (str): Nombre del archivo CSV de salida
        check_availability (bool): Si es True, verifica la disponibilidad de datos antes de procesar

    Returns:
        pd.DataFrame: DataFrame con todos los datos combinados
    """
    # Inicializar Earth Engine
    if not initialize_ee_colab():
        return None

    # Convertir fechas
    start_date_dt = parse_date(start_date)
    end_date_dt = parse_date(end_date)

    # Validar rango de fechas y frecuencia
    is_valid = validate_date_range(start_date_dt, end_date_dt, frequency)
    if not is_valid:
        print("⚠️ Continuando con la extracción, pero puede haber problemas de rendimiento.")

    # Crear punto
    point = ee.Geometry.Point([lon, lat])

    # Visualizar ubicación
    print(f"📍 Ubicación seleccionada: Latitud {lat}, Longitud {lon}")
    visualize_location(lat, lon)

    # Verificar disponibilidad de datos
    if check_availability:
        availability = check_data_availability(point, start_date, end_date)

        # Verificar si hay datos extraíbles
        extractable_count = sum(1 for v in availability.values() if v.get('extractable', False))

        # Si no hay datos extraíbles, sugerir fechas alternativas
        if extractable_count < 3:  # Menos de 3 tipos de datos extraíbles
            good_years = suggest_alternative_dates(point)
            if good_years:
                print("\n⚠️ Continuando con la extracción original, pero es probable que no se encuentren suficientes datos.")
            else:
                print("\n⚠️ Continuando con la extracción, pero es probable que no se encuentren datos.")

    print(f"⏳ Extrayendo datos desde {start_date} hasta {end_date} con frecuencia {frequency}...")

    try:
        # Obtener datos
        print("🔄 Obteniendo datos de albedo...")
        albedo_collection = get_albedo_data(point, start_date, end_date)

        print("🔄 Obteniendo datos de radiación solar...")
        radiation_collection = get_solar_radiation_data(point, start_date, end_date)

        print("🔄 Obteniendo datos de temperatura...")
        temperature_collection = get_temperature_data(point, start_date, end_date)

        print("🔄 Obteniendo datos de viento...")
        # No obtenemos la colección completa aquí, se procesará en chunks

        print("🔄 Obteniendo datos topográficos...")
        topo_data = get_elevation_data(point)

        # Para cobertura terrestre, usar el año medio del rango
        mid_year = start_date_dt.year + (end_date_dt.year - start_date_dt.year) // 2
        print(f"🔄 Obteniendo datos de cobertura terrestre para el año {mid_year}...")
        landcover_data = get_landcover_data(point, mid_year)

        # Procesar datos
        print("🔄 Procesando datos de albedo...")
        albedo_data = process_albedo_data(albedo_collection, point, frequency, start_date_dt, end_date_dt)

        print("🔄 Procesando datos de radiación solar...")
        radiation_data = process_radiation_data(radiation_collection, point, frequency, start_date_dt, end_date_dt)

        print("🔄 Procesando datos de temperatura...")
        temperature_data = process_temperature_data(temperature_collection, point, frequency, start_date_dt, end_date_dt)

        print("🔄 Procesando datos de viento (en chunks para evitar problemas de memoria)...")
        wind_data = process_wind_data_chunked(point, start_date_dt, end_date_dt, frequency)

        print("🔄 Extrayendo datos topográficos...")
        topo_values = extract_static_values(topo_data, point)

        print("🔄 Extrayendo datos de cobertura terrestre...")
        landcover_values = {}
        if landcover_data is not None:
            landcover_values = extract_static_values(landcover_data, point)

        # Combinar todos los datos
        print("🔄 Combinando todos los datos...")
        merged_df = merge_all_data(albedo_data, radiation_data, temperature_data, wind_data, topo_values, landcover_values)

        if merged_df.empty or 'date' not in merged_df.columns or len(merged_df) == 0:
            print("❌ No se encontraron datos para el período y ubicación especificados.")

            # Sugerir fechas alternativas si no se encontraron datos
            good_years = suggest_alternative_dates(point)

            # Crear un DataFrame vacío con columna 'date' para evitar errores
            empty_df = pd.DataFrame({'date': []})
            empty_df.to_csv(output_filename, index=False)
            files.download(output_filename)
            print(f"✅ Se ha generado un archivo CSV vacío: {output_filename}")
            return empty_df

        # Guardar a CSV
        print(f"💾 Guardando datos en {output_filename}...")
        merged_df.to_csv(output_filename, index=False)

        # Descargar archivo
        files.download(output_filename)

        print(f"✅ ¡Completado! Los datos se han guardado y descargado como {output_filename}")

        # Mostrar algunas visualizaciones
        print("\n📊 Visualizaciones de los datos:")

        # Buscar columnas de radiación
        radiation_cols = [col for col in merged_df.columns if 'radiation' in col.lower()]
        if radiation_cols and not merged_df[radiation_cols[0]].isna().all():
            plot_data(merged_df, radiation_cols[0], f'Radiación Solar ({radiation_cols[0]})', 'W/m²')

        # Buscar columnas de temperatura
        if 'temp_mean' in merged_df.columns and not merged_df['temp_mean'].isna().all():
            plot_data(merged_df, 'temp_mean', 'Temperatura Media', '°C')
        elif 'temp_day' in merged_df.columns and not merged_df['temp_day'].isna().all():
            plot_data(merged_df, 'temp_day', 'Temperatura Diurna', '°C')

        # Buscar columnas de albedo
        albedo_cols = [col for col in merged_df.columns if 'bsa' in col.lower() or 'wsa' in col.lower()]
        if albedo_cols and not merged_df[albedo_cols[0]].isna().all():
            plot_data(merged_df, albedo_cols[0], f'Albedo ({albedo_cols[0]})', 'Albedo')

        # Buscar columnas de viento
        if 'wind_speed' in merged_df.columns and not merged_df['wind_speed'].isna().all():
            plot_data(merged_df, 'wind_speed', 'Velocidad del Viento', 'm/s')

        # Mostrar resumen de los datos
        if not merged_df.empty and len(merged_df) > 0:
            print("\n📋 Resumen de los datos extraídos:")
            print(merged_df.describe())

        return merged_df

    except Exception as e:
        print(f"❌ Error durante la extracción de datos: {e}")
        import traceback
        traceback.print_exc()
        # Crear un DataFrame vacío con columna 'date' para evitar errores
        empty_df = pd.DataFrame({'date': []})
        empty_df.to_csv(output_filename, index=False)
        files.download(output_filename)
        print(f"✅ Se ha generado un archivo CSV vacío: {output_filename}")
        return empty_df


In [51]:
data = extract_solar_data_colab(
    lat=37.290019,
    lon=-5.966487,
    start_date='2018-01-01',
    end_date='2018-12-31',
    frequency='monthly',
    output_filename='svq_2018_monthly.csv',
    check_availability=True
)


Earth Engine inicializado correctamente.
✅ Autenticación completada con éxito.
📍 Ubicación seleccionada: Latitud 37.290019, Longitud -5.966487


⚠️ Error al probar extracción de datos: A mapped function's arguments cannot be used in client-side operations
📊 Disponibilidad de datos de albedo: 364 imágenes en la colección, 0 extraíbles para el punto
⚠️ Error al probar extracción de datos: A mapped function's arguments cannot be used in client-side operations
📊 Disponibilidad de datos de radiación solar: 364 imágenes en la colección, 0 extraíbles para el punto
⚠️ Error al probar extracción de datos: A mapped function's arguments cannot be used in client-side operations
📊 Disponibilidad de datos de temperatura: 364 imágenes en la colección, 0 extraíbles para el punto
⚠️ Error al probar extracción de datos: A mapped function's arguments cannot be used in client-side operations
📊 Disponibilidad de datos de viento: 24 imágenes en la colección, 0 extraíbles para el punto
📊 Disponibilidad de datos de elevación: Sí
⚠️ Error al probar extracción de datos: A mapped function's arguments cannot be used in client-side operations
📊 Disponibili

KeyboardInterrupt: 

## Test the API

Test the API by printing the elevation of Mount Everest.

In [ ]:
# Print the elevation of Mount Everest.
dem = ee.Image('USGS/SRTMGL1_003')
xy = ee.Geometry.Point([86.9250, 27.9881])
elev = dem.sample(xy, 30).first().get('elevation').getInfo()
print('Mount Everest elevation (m):', elev)

## Map visualization

`ee.Image` objects can be displayed to notebook output cells. The following two
examples demonstrate displaying a static image and an interactive map.


### Static image

The `IPython.display` module contains the `Image` function, which can display
the results of a URL representing an image generated from a call to the Earth
Engine `getThumbUrl` function. The following cell will display a thumbnail
of the global elevation model.

In [ ]:
# Import the Image function from the IPython.display module.
from IPython.display import Image

# Display a thumbnail of global elevation.
Image(url = dem.updateMask(dem.gt(0))
  .getThumbURL({'min': 0, 'max': 4000, 'dimensions': 512,
                'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}))

### Interactive map

The [geemap](https://github.com/gee-community/geemap)
library can be used to display `ee.Image` objects on an interactive
[ipyleaflet](https://github.com/jupyter-widgets/ipyleaflet) map.

The following cell provides an example of using the `geemap.Map` object to
display an elevation model.

In [ ]:
# Import the geemap library.
import geemap

# Set visualization parameters.
vis_params = {
  'min': 0,
  'max': 4000,
  'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

# Create a map object.
m = geemap.Map(center=[20, 0], zoom=3)

# Add the elevation model to the map object.
m.add_ee_layer(dem.updateMask(dem.gt(0)), vis_params, 'DEM')

# Display the map.
display(m)

## Chart visualization

Some Earth Engine functions produce tabular data that can be plotted by
data visualization packages such as `matplotlib`. The following example
demonstrates the display of tabular data from Earth Engine as a scatter
plot. See [Charting in Colaboratory](https://colab.sandbox.google.com/notebooks/charts.ipynb)
for more information.

In [ ]:
# Import the matplotlib.pyplot module.
import matplotlib.pyplot as plt

# Fetch a Landsat TOA image.
img = ee.Image('LANDSAT/LT05/C02/T1_TOA/LT05_034033_20000913')

# Select Red and NIR bands and sample 500 points.
samp_fc = img.select(['B3','B4']).sample(scale=30, numPixels=500)

# Arrange the sample as a list of lists.
samp_dict = samp_fc.reduceColumns(ee.Reducer.toList().repeat(2), ['B3', 'B4'])
samp_list = ee.List(samp_dict.get('list'))

# Save server-side ee.List as a client-side Python list.
samp_data = samp_list.getInfo()

# Display a scatter plot of Red-NIR sample pairs using matplotlib.
plt.scatter(samp_data[0], samp_data[1], alpha=0.2)
plt.xlabel('Red', fontsize=12)
plt.ylabel('NIR', fontsize=12)
plt.show()